In [ ]:
print("Hello World")

In [ ]:
# --- Install required packages ---
!pip install fastapi uvicorn nest-asyncio pyngrok openpyxl xlrd pandas

# --- Import libraries ---
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse, HTMLResponse, StreamingResponse
from fastapi.middleware.cors import CORSMiddleware
import pandas as pd
from io import BytesIO
import io, os, uuid
from openpyxl.styles import PatternFill, Font
import nest_asyncio
from pyngrok import ngrok

# --- Enable nested asyncio loop for Colab ---
nest_asyncio.apply()

# ---------- HTML for frontend ----------
INDEX_HTML = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Ticket Summary Upload</title>
  <script src="https://cdnjs.cloudflare.com/ajax/libs/xlsx/0.18.5/xlsx.full.min.js"></script>
  <style>
    body { margin: 0; padding: 0; font-family: "Segoe UI", Arial, sans-serif; background: #f4f6f9; color: #333; display: flex; flex-direction: column; min-height: 100vh; }
    .upload-container { background: #fff; padding: 30px; margin: 30px auto 30px auto; border-radius: 12px; box-shadow: 0 6px 20px rgba(0,0,0,0.1); max-width: 700px; width: 90%; text-align: center; }
    h2 { margin-bottom: 25px; color: #444; }
    input[type="file"] { display: block; width: 100%; padding: 12px; border: 2px dashed #ccc; border-radius: 8px; margin-bottom: 20px; cursor: pointer; background: #fafafa; text-align: center; }
    button { display: block; width: 100%; padding: 12px; border: none; border-radius: 8px; background: #0078d7; color: #fff; font-size: 16px; cursor: pointer; transition: background 0.3s ease; margin-bottom: 10px; }
    button:hover { background: #005fa3; }
    .response-container { flex: 1; background: transparent; width: calc(100% - 40px); margin: 0 20px 20px 20px; display: flex; flex-direction: column; }
    #message { text-align: center; font-weight: bold; margin: 15px 0; }
    .error { color: #d9534f; }
    #tableContainer {
    flex: 1;
    overflow-x: auto;
    overflow-y: auto;
    max-height: 500px;
    border: 1px;
    border-radius: 6px;
  }
    table { width: 100%; border-collapse: collapse; margin-top: 10px; }
    th, td { border: 1px solid #ddd; padding: 10px; text-align: center; }
    th { background-color: #0078d7; color: #fff; }
    tr:nth-child(even) { background-color: #f2f2f2; }
    tr:hover { background-color: #e6f2ff; }
  </style>
</head>
<body>

  <div class="upload-container">
    <h2>Upload Excel File for Ticket Summary</h2>
    <input type="file" id="fileInput" accept=".xlsx,.xls" />
    <button onclick="uploadFile()">Upload</button>
    <button id="downloadBtn" style="display:none;" onclick="downloadExcel()">Download Excel</button>
  </div>

  <div class="response-container">
    <p id="message"></p>
    <div id="tableContainer"></div>
  </div>

  <script>
    let tableData = [];

    async function uploadFile() {
      const fileInput = document.getElementById('fileInput');
      const message = document.getElementById('message');
      const tableContainer = document.getElementById('tableContainer');
      const downloadBtn = document.getElementById('downloadBtn');

      if (!fileInput.files.length) {
        message.textContent = "Please select a file!";
        message.className = "error";
        return;
      }

      const file = fileInput.files[0];
      const formData = new FormData();
      formData.append("file", file);

      message.textContent = "Uploading...";
      message.className = "";
      downloadBtn.style.display = "none";

      try {
        const response = await fetch("/upload", { method: "POST", body: formData });
        const result = await response.json();

        if (!result.success) {
          message.textContent = "Error: " + (result.error || "Unknown error");
          message.className = "error";
          tableContainer.innerHTML = "";
          return;
        }

        message.textContent = "Upload successful! Session: " + result.Session;
        message.className = "";

        const data = result.data || [];
        tableData = data;
        if (!data.length) {
          tableContainer.innerHTML = "<p style='text-align:center'>No data found.</p>";
          return;
        }

        let html = "<table><thead><tr>";
        Object.keys(data[0]).forEach(key => html += `<th>${key}</th>`);
        html += "</tr></thead><tbody>";

        data.forEach(row => {
          html += "<tr>";
          Object.values(row).forEach(val => html += `<td>${val}</td>`);
          html += "</tr>";
        });

        html += "</tbody></table>";
        tableContainer.innerHTML = html;

        downloadBtn.style.display = "block";
      } catch (err) {
        message.textContent = "Error: " + err.message;
        message.className = "error";
        tableContainer.innerHTML = "";
        downloadBtn.style.display = "none";
      }
    }

    function downloadExcel() {
      const session = document.getElementById('message').textContent.split('Session: ')[1];
      if (!session) return alert("No session found!");
      const url = `/download/${session}`;
      window.open(url, "_blank");
    }
  </script>

</body>
</html>
"""

# --- Initialize FastAPI app ---
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

session_summaries = {}  # store sessions

@app.get("/", response_class=HTMLResponse)
async def index():
    return HTMLResponse(INDEX_HTML)

# ---------- Excel reading ----------
def read_excel_file(content: bytes, filename: str) -> pd.DataFrame:
    file_stream = BytesIO(content)
    if content[:2] == b'PK':
        return pd.read_excel(file_stream, sheet_name=0, engine="openpyxl")
    elif content[:4] == b'\xd0\xcf\x11\xe0':
        return pd.read_excel(file_stream, sheet_name=0, engine="xlrd")
    else:
        raise ValueError("Unsupported or corrupted Excel file format")

# ---------- Upload endpoint ----------
@app.post("/upload")
async def upload_file(file: UploadFile = File(...)):
    try:
        ext = os.path.splitext(file.filename)[1].lower()
        if ext not in [".xlsx", ".xls"]:
            return JSONResponse({"success": False, "error": "Only .xlsx and .xls files allowed"}, status_code=400)

        content = await file.read()
        df = read_excel_file(content, file.filename)
        df.columns = df.columns.str.strip()
        session_id = str(uuid.uuid4())[:10]

        required_cols = ["Agent", "Status", "Ticket ID"]
        missing = [col for col in required_cols if col not in df.columns]
        if missing:
            return {"success": False, "error": f"Missing required columns: {missing}"}

        summary = (
            df.pivot_table(index="Agent", columns="Status", values="Ticket ID", aggfunc="count", fill_value=0)
            .reset_index()
        )
        summary["Total"] = summary.drop(columns=["Agent"]).sum(axis=1)

        formatted_summary = pd.DataFrame()
        formatted_summary["Agent"] = summary["Agent"]
        for col in summary.columns:
            if col in ["Agent", "Total"]:
                continue
            formatted_summary[col] = summary[col]
            formatted_summary[f"{col} %"] = summary.apply(
                lambda row: f"{int(round((row[col]/row['Total']*100)))}%" if row["Total"] > 0 else "0%",
                axis=1
            )
        formatted_summary["Total"] = summary["Total"]
        data = formatted_summary.to_dict(orient="records")
        session_summaries[session_id] = formatted_summary

        return {"success": True, "Session": session_id, "data": data}
    except Exception as e:
        return JSONResponse({"success": False, "error": str(e)}, status_code=400)

# ---------- Download endpoint ----------
@app.get("/download/{session_id}")
async def download_excel(session_id: str):
    try:
        if session_id not in session_summaries:
            return {"success": False, "error": "Session not found"}

        summary = session_summaries[session_id]
        if summary.empty:
            return {"success": False, "error": "No data to download"}

        output = io.BytesIO()
        with pd.ExcelWriter(output, engine="openpyxl") as writer:
            summary.to_excel(writer, index=False, sheet_name="Summary")
            ws = writer.sheets["Summary"]
            header_fill = PatternFill(start_color="0078D7", end_color="0078D7", fill_type="solid")
            header_font = Font(color="FFFFFF", bold=True)
            for cell in ws[1]:
                cell.fill = header_fill
                cell.font = header_font
        output.seek(0)
        return StreamingResponse(
            output,
            media_type="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
            headers={"Content-Disposition": "attachment; filename=Ticket_summary.xlsx"},
        )
    except Exception as e:
        return {"success": False, "error": str(e)}

# ---------- Start server with ngrok ----------
port = 8000
public_url = ngrok.connect(port)
print(f"🚀 FastAPI is running! Open the link: {public_url}")

import uvicorn
uvicorn.run(app, host="0.0.0.0", port=port)
